# 03. Data Update & Quality Assurance

---

## 목표
1. 기존 데이터(2018-11-29 ~ 2023-11-29) 품질 검증
2. 신규 데이터(2023-11-30 ~ 현재) 수집 및 병합
3. 섹터/산업 메타데이터 최신화
4. 파생변수 재계산 (기존 + RSI, Bollinger Bands)

## 주요 고려사항
- Adjusted Close(수정 종가) 정합성 검증
- 상폐/티커 변경 종목 예외 처리
- 기업별 관측일 불일치 분석
- OHLC 이상치 검출

---

## Section 1. 환경 설정 & 기존 데이터 로드

In [ ]:
# Library Imports
import pandas as pd
import numpy as np
import yfinance as yf
import os
import json
import time
from pathlib import Path
from datetime import datetime, timedelta
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

# 시각화 설정
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [2]:
# 경로 설정
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / 'Data_set'
QA_DIR = DATA_DIR / 'QA'
LOG_DIR = DATA_DIR / 'Logs'

# 디렉토리 생성
QA_DIR.mkdir(exist_ok=True)
LOG_DIR.mkdir(exist_ok=True)

In [3]:
# 기존 데이터 로드
df_existing = pd.read_csv(DATA_DIR / 'stock_daily_master.csv')
df_existing['Date'] = pd.to_datetime(df_existing['Date'])

print(f"기존 데이터 Shape: {df_existing.shape}")
print(f"기간: {df_existing['Date'].min().strftime('%Y-%m-%d')} ~ {df_existing['Date'].max().strftime('%Y-%m-%d')}")
print(f"기업 수: {df_existing['Company'].nunique()}")
print(f"\n컬럼 목록:")
print(df_existing.columns.tolist())

기존 데이터 Shape: (602962, 26)
기간: 2018-11-29 ~ 2023-11-29
기업 수: 491

컬럼 목록:
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Company', 'Sector', 'Daily_Return', 'Cum_Return', 'MA_5', 'MA_20', 'MA_60', 'Volatility_20d', 'Cum_Max', 'Drawdown', 'MDD', 'Vol_MA_20', 'Vol_Ratio', 'Vol_Std_20', 'Vol_Z_Score', 'Prev_Close', 'Gap', 'Gap_Pct']


In [4]:
# 티커 목록 추출
tickers = df_existing['Company'].unique().tolist()
print(f"총 티커 수: {len(tickers)}")
print(f"샘플: {tickers[:20]}")

총 티커 수: 491
샘플: ['A', 'AAPL', 'ABBV', 'ABEV', 'ABNB', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEM', 'AEP', 'AFL', 'AIG', 'AJG', 'ALC']


---

## Section 2. 데이터 품질 검증 (QA)

### 검증 항목
1. 기업별 관측일 수 분석 (동일 기간 비교 가능 여부)
2. OHLC 이상치 검증
3. Dividends & Stock Splits 이벤트 분석
4. 수익률 결측치 처리 검증

### 2.1 기업별 관측일 수 분석

In [5]:
import pandas as pd

df_existing['Date'] = pd.to_datetime(df_existing['Date']) 

# 기업별 관측일 통계
company_stats = df_existing.groupby('Company').agg({
    'Date': ['count', 'min', 'max']
}).reset_index()

# 컬럼명 재설정 (MultiIndex Flattening)
company_stats.columns = ['Company', 'Obs_Count', 'Start_Date', 'End_Date']

# 날짜 객체끼리의 연산이므로 Timedelta가 생성되고 .dt.days 사용 가능
company_stats['Days_Span'] = (company_stats['End_Date'] - company_stats['Start_Date']).dt.days

print("=" * 60)
print("기업별 관측일 수 통계")
print("=" * 60)
print(company_stats[['Obs_Count', 'Days_Span']].describe())
print(f"\n기대 관측일 수 (5년): ~1,258일 (252 거래일 x 5년)")

# 데이터 확인 (상위 5개)
print(company_stats.head())

기업별 관측일 수 통계
         Obs_Count    Days_Span
count   491.000000   491.000000
mean   1228.028513  1782.358452
std     142.004194   206.450908
min      54.000000    76.000000
25%    1258.000000  1826.000000
50%    1258.000000  1826.000000
75%    1258.000000  1826.000000
max    1258.000000  1826.000000

기대 관측일 수 (5년): ~1,258일 (252 거래일 x 5년)
  Company  Obs_Count Start_Date   End_Date  Days_Span
0       A       1258 2018-11-29 2023-11-29       1826
1    AAPL       1258 2018-11-29 2023-11-29       1826
2    ABBV       1258 2018-11-29 2023-11-29       1826
3    ABEV       1258 2018-11-29 2023-11-29       1826
4    ABNB        747 2020-12-10 2023-11-29       1084


In [6]:
# 동일 기간 비교 가능 기업 분류
# 기준: 최소 관측일의 80% 이상 보유
max_obs = company_stats['Obs_Count'].max()
threshold = max_obs * 0.8

# 전체 기간 데이터 보유 기업
full_period_mask = company_stats['Obs_Count'] >= threshold
full_period_companies = company_stats[full_period_mask]['Company'].tolist()
partial_period_companies = company_stats[~full_period_mask]['Company'].tolist()

print("=" * 60)
print("기업 분류 결과")
print("=" * 60)
print(f"동일 기간 비교 가능 기업: {len(full_period_companies)}개 ({len(full_period_companies)/len(tickers)*100:.1f}%)")
print(f"개별 분석 대상 기업: {len(partial_period_companies)}개 ({len(partial_period_companies)/len(tickers)*100:.1f}%)")
print(f"\n기준: 최대 관측일({max_obs})의 80% = {threshold:.0f}일 이상")

기업 분류 결과
동일 기간 비교 가능 기업: 471개 (95.9%)
개별 분석 대상 기업: 20개 (4.1%)

기준: 최대 관측일(1258)의 80% = 1006일 이상


In [7]:
# 개별 분석 대상 기업 상세
partial_stats = company_stats[~full_period_mask].sort_values('Obs_Count')

print("개별 분석 대상 기업 (관측일 적은 순):")
print(partial_stats[['Company', 'Obs_Count', 'Start_Date', 'End_Date']].head(30).to_string(index=False))

개별 분석 대상 기업 (관측일 적은 순):
Company  Obs_Count Start_Date   End_Date
    ARM         54 2023-09-14 2023-11-29
   KVUE        145 2023-05-04 2023-11-29
   GEHC        240 2022-12-15 2023-11-29
   MBLY        275 2022-10-26 2023-11-29
    HLN        341 2022-07-25 2023-11-29
    CEG        469 2022-01-19 2023-11-29
     NU        496 2021-12-09 2023-11-29
    GFS        525 2021-10-28 2023-11-29
   COIN        663 2021-04-14 2023-11-29
   CPNG        686 2021-03-11 2023-11-29
   RBLX        687 2021-03-10 2023-11-29
    SYM        688 2021-03-09 2023-11-29
   ABNB        747 2020-12-10 2023-11-29
   DASH        748 2020-12-09 2023-11-29
   PLTR        797 2020-09-30 2023-11-29
   SNOW        807 2020-09-16 2023-11-29
   BEKE        830 2020-08-13 2023-11-29
     LI        840 2020-07-30 2023-11-29
   CARR        932 2020-03-19 2023-11-29
   OTIS        932 2020-03-19 2023-11-29


### 2.2 OHLC 이상치 검증

In [8]:
# OHLC 이상치 검증 함수
def validate_ohlc(df):
    """OHLC 데이터 이상치 검증"""
    anomalies = []
    
    # 1. 0값 또는 음수 검출
    for col in ['Open', 'High', 'Low', 'Close']:
        zero_or_neg = df[df[col] <= 0]
        if len(zero_or_neg) > 0:
            for _, row in zero_or_neg.iterrows():
                anomalies.append({
                    'Company': row['Company'],
                    'Date': row['Date'],
                    'Type': f'{col} <= 0',
                    'Value': row[col]
                })
    
    # 2. High < Low 역전
    inverted = df[df['High'] < df['Low']]
    for _, row in inverted.iterrows():
        anomalies.append({
            'Company': row['Company'],
            'Date': row['Date'],
            'Type': 'High < Low',
            'Value': f"H:{row['High']:.2f}, L:{row['Low']:.2f}"
        })
    
    # 3. Open/Close가 High/Low 범위 밖
    out_of_range = df[(df['Open'] > df['High']) | (df['Open'] < df['Low']) |
                      (df['Close'] > df['High']) | (df['Close'] < df['Low'])]
    for _, row in out_of_range.iterrows():
        anomalies.append({
            'Company': row['Company'],
            'Date': row['Date'],
            'Type': 'Open/Close out of H/L range',
            'Value': f"O:{row['Open']:.2f}, H:{row['High']:.2f}, L:{row['Low']:.2f}, C:{row['Close']:.2f}"
        })
    
    # 4. 극단적 일간 변동 (>50%)
    df_temp = df.copy()
    df_temp['Daily_Change'] = df_temp.groupby('Company')['Close'].pct_change().abs()
    extreme_change = df_temp[df_temp['Daily_Change'] > 0.5]
    for _, row in extreme_change.iterrows():
        anomalies.append({
            'Company': row['Company'],
            'Date': row['Date'],
            'Type': 'Extreme daily change (>50%)',
            'Value': f"{row['Daily_Change']*100:.1f}%"
        })
    
    return pd.DataFrame(anomalies)

# 검증 실행
anomaly_df = validate_ohlc(df_existing)

print("=" * 60)
print("OHLC 이상치 검증 결과")
print("=" * 60)
if len(anomaly_df) == 0:
    print("이상치 없음")
else:
    print(f"총 이상치: {len(anomaly_df)}건")
    print(f"\n유형별 집계:")
    print(anomaly_df['Type'].value_counts())

OHLC 이상치 검증 결과
총 이상치: 42건

유형별 집계:
Type
Open/Close out of H/L range    30
Extreme daily change (>50%)    12
Name: count, dtype: int64


In [9]:
# 이상치 상세
if len(anomaly_df) > 0:
    print("\n이상치 상세 :")
    print(anomaly_df.head(42).to_string(index=False))


이상치 상세 :
Company       Date                        Type                                  Value
      A 2023-11-29 Open/Close out of H/L range O:125.59, H:127.42, L:125.66, C:127.01
    AEM 2023-11-29 Open/Close out of H/L range     O:53.69, H:53.48, L:52.65, C:52.96
    BMO 2023-11-29 Open/Close out of H/L range     O:79.86, H:80.82, L:79.87, C:80.63
   BSBR 2023-11-29 Open/Close out of H/L range         O:6.20, H:6.28, L:6.21, C:6.26
     DB 2023-11-29 Open/Close out of H/L range     O:12.27, H:12.41, L:12.28, C:12.38
    ELP 2019-06-06 Open/Close out of H/L range         O:4.01, H:4.02, L:3.98, C:3.98
    ENB 2023-11-29 Open/Close out of H/L range     O:34.35, H:34.63, L:34.38, C:34.54
   FERG 2020-04-27 Open/Close out of H/L range     O:61.30, H:61.30, L:61.30, C:61.30
    FNV 2023-11-29 Open/Close out of H/L range O:117.50, H:117.40, L:114.88, C:115.07
    GIB 2023-11-29 Open/Close out of H/L range  O:101.35, H:101.03, L:99.86, C:100.04
     HD 2023-11-29 Open/Close out of H/L ran

### 2.3 데이터 정제 (Data Cleaning)

**목적**: OHLC 논리 오류 수정 및 극단 변화 플래그 추가

**처리 항목**:
1. OHLC 범위 오류 수정 (Open/Close out of H/L range)
2. 극단적 일간 변동(>50%) 플래그 추가 (삭제하지 않음)
3. 부동소수점 오차 허용 (tolerance=0.01)

In [10]:
def clean_ohlc_violations(df, tolerance=0.01):
    """
    OHLC 범위 오류 수정
    
    Args:
        df: 원본 데이터프레임
        tolerance: 부동소수점 오차 허용값 (기본 0.01 = 1센트)
    
    Returns:
        df_cleaned: 정제된 데이터프레임
        cleaning_log: 수정 내역 로그
    """
    df_cleaned = df.copy()
    cleaning_log = []
    
    # 1. Open > High 수정
    mask_open_high = df_cleaned['Open'] > (df_cleaned['High'] + tolerance)
    if mask_open_high.sum() > 0:
        for idx in df_cleaned[mask_open_high].index:
            row = df_cleaned.loc[idx]
            cleaning_log.append({
                'Index': idx,
                'Company': row['Company'],
                'Date': row['Date'],
                'Type': 'Open > High',
                'Original_High': row['High'],
                'Original_Open': row['Open'],
                'Corrected_High': row['Open']
            })
        df_cleaned.loc[mask_open_high, 'High'] = df_cleaned.loc[mask_open_high, 'Open']
    
    # 2. Open < Low 수정
    mask_open_low = df_cleaned['Open'] < (df_cleaned['Low'] - tolerance)
    if mask_open_low.sum() > 0:
        for idx in df_cleaned[mask_open_low].index:
            row = df_cleaned.loc[idx]
            cleaning_log.append({
                'Index': idx,
                'Company': row['Company'],
                'Date': row['Date'],
                'Type': 'Open < Low',
                'Original_Low': row['Low'],
                'Original_Open': row['Open'],
                'Corrected_Low': row['Open']
            })
        df_cleaned.loc[mask_open_low, 'Low'] = df_cleaned.loc[mask_open_low, 'Open']
    
    # 3. Close > High 수정
    mask_close_high = df_cleaned['Close'] > (df_cleaned['High'] + tolerance)
    if mask_close_high.sum() > 0:
        for idx in df_cleaned[mask_close_high].index:
            row = df_cleaned.loc[idx]
            cleaning_log.append({
                'Index': idx,
                'Company': row['Company'],
                'Date': row['Date'],
                'Type': 'Close > High',
                'Original_High': row['High'],
                'Original_Close': row['Close'],
                'Corrected_High': row['Close']
            })
        df_cleaned.loc[mask_close_high, 'High'] = df_cleaned.loc[mask_close_high, 'Close']
    
    # 4. Close < Low 수정
    mask_close_low = df_cleaned['Close'] < (df_cleaned['Low'] - tolerance)
    if mask_close_low.sum() > 0:
        for idx in df_cleaned[mask_close_low].index:
            row = df_cleaned.loc[idx]
            cleaning_log.append({
                'Index': idx,
                'Company': row['Company'],
                'Date': row['Date'],
                'Type': 'Close < Low',
                'Original_Low': row['Low'],
                'Original_Close': row['Close'],
                'Corrected_Low': row['Close']
            })
        df_cleaned.loc[mask_close_low, 'Low'] = df_cleaned.loc[mask_close_low, 'Close']
    
    cleaning_log_df = pd.DataFrame(cleaning_log)
    
    return df_cleaned, cleaning_log_df


def add_extreme_change_flag(df, threshold=0.5):
    """
    극단적 일간 변동 플래그 추가 (데이터 삭제하지 않음)
    
    Args:
        df: 데이터프레임
        threshold: 극단 변화 기준 (기본 0.5 = 50%)
    
    Returns:
        df: 플래그가 추가된 데이터프레임
    """
    df = df.copy()
    
    # 기업별로 일간 수익률 계산 (임시)
    df['Daily_Change_Temp'] = df.groupby('Company')['Close'].pct_change().abs()
    
    # 극단 변화 플래그
    df['Is_Extreme_Change'] = df['Daily_Change_Temp'] > threshold
    
    # 임시 컬럼 제거
    df = df.drop(columns=['Daily_Change_Temp'])
    
    return df


def validate_ohlc_with_tolerance(df, tolerance=0.01):
    """
    부동소수점 오차를 허용한 OHLC 검증
    
    Args:
        df: 데이터프레임
        tolerance: 허용 오차 (기본 0.01)
    
    Returns:
        anomaly_df: 이상치 데이터프레임
    """
    anomalies = []
    
    # 1. 0값 또는 음수 검출
    for col in ['Open', 'High', 'Low', 'Close']:
        zero_or_neg = df[df[col] <= 0]
        if len(zero_or_neg) > 0:
            for _, row in zero_or_neg.iterrows():
                anomalies.append({
                    'Company': row['Company'],
                    'Date': row['Date'],
                    'Type': f'{col} <= 0',
                    'Value': row[col]
                })
    
    # 2. High < Low 역전 (tolerance 적용)
    inverted = df[df['High'] < (df['Low'] - tolerance)]
    for _, row in inverted.iterrows():
        anomalies.append({
            'Company': row['Company'],
            'Date': row['Date'],
            'Type': 'High < Low',
            'Value': f"H:{row['High']:.2f}, L:{row['Low']:.2f}"
        })
    
    # 3. Open/Close가 High/Low 범위 밖 (tolerance 적용)
    out_of_range = df[
        (df['Open'] > df['High'] + tolerance) | 
        (df['Open'] < df['Low'] - tolerance) |
        (df['Close'] > df['High'] + tolerance) | 
        (df['Close'] < df['Low'] - tolerance)
    ]
    for _, row in out_of_range.iterrows():
        anomalies.append({
            'Company': row['Company'],
            'Date': row['Date'],
            'Type': 'Open/Close out of H/L range',
            'Value': f"O:{row['Open']:.2f}, H:{row['High']:.2f}, L:{row['Low']:.2f}, C:{row['Close']:.2f}"
        })
    
    return pd.DataFrame(anomalies)

print("데이터 정제 함수 정의 완료")

데이터 정제 함수 정의 완료


In [11]:
# 1. OHLC 범위 오류 수정
print("OHLC 범위 오류 수정")

df_cleaned, cleaning_log = clean_ohlc_violations(df_existing, tolerance=0.01)

print(f"수정된 건수: {len(cleaning_log)}건")

if len(cleaning_log) > 0:
    print(f"\n수정 유형별 집계:")
    print(cleaning_log['Type'].value_counts())
    print(f"\n수정 상세:")
    print(cleaning_log.head(10).to_string(index=False))

OHLC 범위 오류 수정
수정된 건수: 23건

수정 유형별 집계:
Type
Open < Low     14
Open > High     9
Name: count, dtype: int64

수정 상세:
 Index Company       Date        Type  Original_High  Original_Open  Corrected_High  Original_Low  Corrected_Low
 18358     AEM 2023-11-29 Open > High      53.479900      53.689999       53.689999           NaN            NaN
226963     FNV 2023-11-29 Open > High     117.400002     117.500000      117.500000           NaN            NaN
235276     GIB 2023-11-29 Open > High     101.030899     101.349998      101.349998           NaN            NaN
254146      HD 2023-11-29 Open > High     313.890015     314.059998      314.059998           NaN            NaN
256662     HEI 2023-11-29 Open > High     170.899506     171.009995      171.009995           NaN            NaN
320888     LMT 2023-11-29 Open > High     449.429993     449.540009      449.540009           NaN            NaN
529029     TRI 2023-11-29 Open > High     140.360001     140.429993      140.429993           Na

In [12]:
# 2. 극단적 변화 플래그 추가

df_cleaned = add_extreme_change_flag(df_cleaned, threshold=0.5)

extreme_count = df_cleaned['Is_Extreme_Change'].sum()
print(f"\n플래그 추가 완료!")
print(f"극단 변화 건수: {extreme_count}건 (삭제하지 않고 보존)")

if extreme_count > 0:
    print(f"\n극단 변화 기업 (상위 10건):")
    extreme_cases = df_cleaned[df_cleaned['Is_Extreme_Change']]
    print(extreme_cases[['Company', 'Date', 'Close']].head(10).to_string(index=False))


플래그 추가 완료!
극단 변화 건수: 12건 (삭제하지 않고 보존)

극단 변화 기업 (상위 10건):
Company       Date      Close
   BEKE 2022-03-16  14.731359
   BNTX 2020-03-17  65.729401
    CVE 2020-03-09   2.652992
  FCNCA 2023-03-27 894.600281
    OXY 2020-03-09  12.256660
    PCG 2019-01-14   8.380000
    PCG 2019-01-24  13.950000
    PDD 2022-03-16  42.619999
   SNAP 2022-02-04  38.910000
    SYM 2022-06-08  20.070000


In [13]:
# 3. 정제 후 재검증 (tolerance 적용)
print("\n" + "=" * 60)
print("정제 후 OHLC 재검증 (tolerance=0.01)")
print("=" * 60)

anomaly_after = validate_ohlc_with_tolerance(df_cleaned, tolerance=0.01)

print(f"\n재검증 결과:")
if len(anomaly_after) == 0:
    print("OHLC 이상치 없음")
else:
    print(f"   남은 이상치: {len(anomaly_after)}건")
    print(f"\n유형별 집계:")
    print(anomaly_after['Type'].value_counts())
    print(f"\n상세:")
    print(anomaly_after.head(10).to_string(index=False))


정제 후 OHLC 재검증 (tolerance=0.01)



재검증 결과:
OHLC 이상치 없음


In [ ]:
# 4. Before/After 비교 리포트
print("\n" + "=" * 60)
print("데이터 정제 비교")
print("=" * 60)

print(f"\n{'항목':<30} {'Before':<15} {'After':<15} {'개선':<10}")
print("-" * 70)

# OHLC 범위 오류
before_ohlc = len(anomaly_df[anomaly_df['Type'] == 'Open/Close out of H/L range'])
# Handle empty DataFrame case
if len(anomaly_after) == 0 or 'Type' not in anomaly_after.columns:
    after_ohlc = 0
else:
    after_ohlc = len(anomaly_after[anomaly_after['Type'] == 'Open/Close out of H/L range'])
print(f"{'OHLC 범위 오류':<30} {before_ohlc:<15} {after_ohlc:<15} {'✅' if after_ohlc == 0 else '⚠️'}")

# 극단 변화 (플래그로 전환)
before_extreme = len(anomaly_df[anomaly_df['Type'] == 'Extreme daily change (>50%)'])
after_extreme = df_cleaned['Is_Extreme_Change'].sum()
print(f"{'극단 변화 (>50%)':<30} {f'{before_extreme} (오류)':<15} {f'{after_extreme} (플래그)':<15} {'✅'}")

# 전체 데이터
print(f"{'전체 데이터 행 수':<30} {len(df_existing):<15} {len(df_cleaned):<15} {'✅'}")

print("\n" + "=" * 60)
print("데이터 정제 완료!")
print("=" * 60)


데이터 정제 비교

항목                             Before          After           개선        
----------------------------------------------------------------------
OHLC 범위 오류                     30              0               ✅
극단 변화 (>50%)                   12 (오류)         12 (플래그)        ✅
전체 데이터 행 수                     602962          602962          ✅

✅ 데이터 정제 완료!


In [15]:
# 5. 정제 로그 저장
if len(cleaning_log) > 0:
    cleaning_log.to_csv(QA_DIR / 'ohlc_cleaning_log.csv', index=False)
    print(f"\n정제 로그 저장: {QA_DIR / 'ohlc_cleaning_log.csv'}")

# 기존 데이터를 정제된 데이터로 교체
df_existing = df_cleaned.copy()
print(f"df_existing을 정제된 데이터로 업데이트 완료")


정제 로그 저장: /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/QA/ohlc_cleaning_log.csv
df_existing을 정제된 데이터로 업데이트 완료


### 2.4 Dividends & Stock Splits 이벤트 분석

In [16]:
# 배당 이벤트 분석
dividend_events = df_existing[df_existing['Dividends'] > 0].copy()
dividend_companies = dividend_events['Company'].unique()

print("=" * 60)
print("배당 (Dividends) 이벤트 분석")
print("=" * 60)
print(f"배당 지급 기업 수: {len(dividend_companies)}개 ({len(dividend_companies)/len(tickers)*100:.1f}%)")
print(f"총 배당 이벤트 수: {len(dividend_events)}건")

# 기업별 배당 횟수
dividend_freq = dividend_events.groupby('Company').size().sort_values(ascending=False)
print(f"\n기업별 배당 횟수 (상위 10):")
print(dividend_freq.head(10))

배당 (Dividends) 이벤트 분석
배당 지급 기업 수: 385개 (78.4%)
총 배당 이벤트 수: 6857건

기업별 배당 횟수 (상위 10):
Company
ITUB    80
BBD     69
O       60
COP     28
EOG     26
PCAR    25
CME     25
BSBR    22
NOC     21
FAST    21
dtype: int64


In [17]:
# 주식 분할 이벤트 분석
split_events = df_existing[df_existing['Stock Splits'] > 0].copy()
split_companies = split_events['Company'].unique()

print("=" * 60)
print("주식 분할 분석")
print("=" * 60)
print(f"주식 분할 기업 수: {len(split_companies)}개 ({len(split_companies)/len(tickers)*100:.1f}%)")
print(f"총 분할 수: {len(split_events)}건")

if len(split_events) > 0:
    print(f"\n주식 분할 상세:")
    print(split_events[['Company', 'Date', 'Stock Splits']].to_string(index=False))

주식 분할 분석
주식 분할 기업 수: 61개 (12.4%)
총 분할 수: 71건

주식 분할 상세:
Company       Date  Stock Splits
   AAPL 2020-08-31        4.0000
   AMZN 2022-06-06       20.0000
   ANET 2021-11-18        4.0000
    APH 2021-03-05        2.0000
    BBD 2019-04-01        1.2000
    BDX 2022-04-01        1.0250
    BHP 2022-06-02        1.1210
     BN 2020-04-02        1.5000
     BN 2022-12-12        1.2370
     CM 2022-05-16        2.0000
    CNC 2019-02-07        2.0000
    CNQ 2022-08-22        1.0210
     CP 2021-05-14        5.0000
   CPRT 2022-11-04        2.0000
   CPRT 2023-08-22        2.0000
   CSGP 2021-06-28       10.0000
    CSX 2021-06-29        3.0000
     DD 2019-04-02        1.4870
     DD 2019-06-03        0.4725
   DELL 2018-12-28        1.8060
   DELL 2021-11-02        1.9730
    DHR 2023-10-02        1.1280
    DTE 2021-07-01        1.1750
   DXCM 2022-06-13        4.0000
    ELP 2021-03-18       10.0000
    ELP 2021-04-28        0.2000
     EW 2020-06-01        3.0000
    EXC 2022-02-02  

In [18]:
# 이벤트(배당, 주식 분할) 기업 목록 저장용 데이터프레임 생성
event_summary = []

for company in tickers:
    div_count = len(dividend_events[dividend_events['Company'] == company])
    split_count = len(split_events[split_events['Company'] == company])
    
    if div_count > 0 or split_count > 0:
        event_summary.append({
            'Company': company,
            'Dividend_Count': div_count,
            'Split_Count': split_count,
            'Has_Dividend': div_count > 0,
            'Has_Split': split_count > 0
        })

event_df = pd.DataFrame(event_summary)
print(f"\n배당, 주식 분할 보유 기업 수: {len(event_df)}개")


배당, 주식 분할 보유 기업 수: 399개

### 2.4 수익률 결측치 처리 검증

In [19]:
# 각 기업 첫 거래일의 Daily_Return 검증
first_day_returns = df_existing.groupby('Company').first()['Daily_Return']

# NaN인 경우 (정상)
nan_first_day = first_day_returns.isna().sum()
# 0인 경우 (의심)
zero_first_day = (first_day_returns == 0).sum()
# 값이 있는 경우 (오류 가능성)
has_value_first_day = first_day_returns.notna().sum() - zero_first_day

print("=" * 60)
print("수익률 결측치 처리 검증")
print("=" * 60)
print(f"첫 거래일 Daily_Return = NaN: {nan_first_day}개 {'정상' if nan_first_day == len(tickers) else ''}")
print(f"첫 거래일 Daily_Return = 0: {zero_first_day}개 {'검토 필요' if zero_first_day > 0 else ''}")
print(f"첫 거래일 Daily_Return = 기타값: {has_value_first_day}개 {'검토 필요' if has_value_first_day > 0 else ''}")

수익률 결측치 처리 검증
첫 거래일 Daily_Return = NaN: 0개 
첫 거래일 Daily_Return = 0: 4개 검토 필요
첫 거래일 Daily_Return = 기타값: 487개 검토 필요


In [20]:
# Daily_Return이 0인데 Close 변화가 있는 경우 (오류)
df_check = df_existing.copy()
df_check['Prev_Close_Calc'] = df_check.groupby('Company')['Close'].shift(1)
df_check['Close_Changed'] = (df_check['Close'] != df_check['Prev_Close_Calc']) & df_check['Prev_Close_Calc'].notna()

suspicious_zeros = df_check[(df_check['Daily_Return'] == 0) & df_check['Close_Changed']]

print(f"\nDaily_Return=0 but Close changed: {len(suspicious_zeros)}건")
if len(suspicious_zeros) > 0:
    print("부적절한 0 대입 의심")
    print(suspicious_zeros[['Company', 'Date', 'Close', 'Prev_Close_Calc', 'Daily_Return']].head(10))
else:
    print("부적절한 0 대입 없음")


Daily_Return=0 but Close changed: 0건
부적절한 0 대입 없음


In [21]:
# 연속 NaN 패턴 (거래 정지 등)
def find_consecutive_nans(group):
    """연속 NaN 구간 찾기"""
    is_nan = group['Daily_Return'].isna()
    # 첫날 NaN 제외
    is_nan.iloc[0] = False
    
    consecutive = (is_nan != is_nan.shift()).cumsum()
    nan_groups = group[is_nan].groupby(consecutive[is_nan]).size()
    
    if len(nan_groups) > 0 and nan_groups.max() > 5:  # 5일 이상 연속 NaN
        return nan_groups.max()
    return 0

consecutive_nans = df_existing.groupby('Company').apply(find_consecutive_nans)
companies_with_gaps = consecutive_nans[consecutive_nans > 0]

print(f"\n5일 이상 연속 결측 기업: {len(companies_with_gaps)}개")
if len(companies_with_gaps) > 0:
    print(companies_with_gaps.sort_values(ascending=False).head(10))


5일 이상 연속 결측 기업: 0개

---

## Section 3. 티커 메타데이터 수집 (섹터/산업 최신화)

In [ ]:
def fetch_ticker_metadata(ticker_list, batch_size=50, max_retries=3):
    """
    yfinance를 사용하여 티커별 섹터/산업 정보 수집
    
    Args:
        ticker_list: 티커 목록
        batch_size: 배치 크기 (API 부하 방지)
        max_retries: 최대 재시도 횟수
    
    Returns:
        dict: {ticker: {'Sector': ..., 'Industry': ...}}
    """
    metadata = {}
    failed_tickers = []
    
    # 배치 분할
    chunks = [ticker_list[i:i + batch_size] for i in range(0, len(ticker_list), batch_size)]
    
    for chunk in tqdm(chunks, desc="Fetching Metadata"):
        tickers_str = ' '.join(chunk)
        
        try:
            tickers_obj = yf.Tickers(tickers_str)
            
            for symbol in chunk:
                for attempt in range(max_retries):
                    try:
                        info = tickers_obj.tickers[symbol].info
                        
                        sector = info.get('sector', 'Unknown')
                        industry = info.get('industry', 'Unknown')
                        
                        # Unknown이면 개별 호출 재시도
                        if sector == 'Unknown':
                            single = yf.Ticker(symbol)
                            sector = single.info.get('sector', 'Unknown')
                            industry = single.info.get('industry', 'Unknown')
                        
                        metadata[symbol] = {
                            'Sector': sector,
                            'Industry': industry
                        }
                        break
                        
                    except Exception as e:
                        if attempt == max_retries - 1:
                            failed_tickers.append(symbol)
                            metadata[symbol] = {'Sector': 'Error', 'Industry': 'Error'}
                        time.sleep(0.5)
                
                time.sleep(0.05)  # Rate limit 방지
                
        except Exception as e:
            print(f"Chunk Error: {e}")
            for symbol in chunk:
                failed_tickers.append(symbol)
                metadata[symbol] = {'Sector': 'Error', 'Industry': 'Error'}
    
    return metadata, failed_tickers

메타데이터 수집 함수 정의 완료


In [ ]:
# 메타데이터 수집 실행
print(f"대상 티커 수: {len(tickers)}개")
print(f"예상 소요 시간: ~{len(tickers) * 0.1 / 60:.1f}분\n")

metadata, failed_tickers = fetch_ticker_metadata(tickers)

print(f"\n수집 완료")
print(f"성공: {len(metadata) - len(failed_tickers)}개")
print(f"실패: {len(failed_tickers)}개")

if failed_tickers:
    print(f"\n실패 티커: {failed_tickers[:20]}{'...' if len(failed_tickers) > 20 else ''}")

섹터/산업 메타데이터 수집 시작...
대상 티커 수: 491개
예상 소요 시간: ~0.8분




Fetching Metadata:   0%|          | 0/10 [00:00<?, ?it/s]

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ANSS"}}}


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ANSS"}}}



Fetching Metadata:  10%|█         | 1/10 [00:25<03:53, 25.93s/it]

...


Fetching Metadata:  80%|████████  | 8/10 [03:16<00:48, 24.33s/it]


Fetching Metadata:  90%|█████████ | 9/10 [03:41<00:24, 24.54s/it]


Fetching Metadata: 100%|██████████| 10/10 [04:00<00:00, 23.03s/it]


Fetching Metadata: 100%|██████████| 10/10 [04:00<00:00, 24.08s/it]


수집 완료!
성공: 491개
실패: 0개


In [24]:
# 메타데이터 데이터프레임 변환
metadata_df = pd.DataFrame.from_dict(metadata, orient='index').reset_index()
metadata_df.columns = ['Company', 'Sector_New', 'Industry_New']

# 섹터별 분포 확인
print("=" * 60)
print("최신 섹터 분포")
print("=" * 60)
print(metadata_df['Sector_New'].value_counts())

최신 섹터 분포
Sector_New
Financial Services        82
Technology                77
Healthcare                57
Industrials               56
Consumer Cyclical         45
Energy                    38
Consumer Defensive        31
Communication Services    27
Basic Materials           26
Utilities                 24
Real Estate               19
Unknown                    9
Name: count, dtype: int64


---

## Section 4. 신규 가격 데이터 수집 (2023-11-30 ~ 현재)

In [ ]:
def fetch_price_data(ticker_list, start_date, end_date, batch_size=50):
    """
    yfinance를 사용하여 가격 데이터 수집
    
    Args:
        ticker_list: 티커 목록
        start_date: 시작일 (YYYY-MM-DD)
        end_date: 종료일 (YYYY-MM-DD)
        batch_size: 배치 크기
    
    Returns:
        DataFrame: OHLCV 데이터
    """
    all_data = []
    failed_tickers = []
    
    # 배치 분할
    chunks = [ticker_list[i:i + batch_size] for i in range(0, len(ticker_list), batch_size)]
    
    for chunk in tqdm(chunks, desc="Fetching Price Data"):
        tickers_str = ' '.join(chunk)
        
        try:
            # 배치 다운로드
            data = yf.download(
                tickers_str,
                start=start_date,
                end=end_date,
                group_by='ticker',
                auto_adjust=True,  # Adjusted Close 사용
                progress=False
            )
            
            if len(chunk) == 1:
                # 단일 티커인 경우 구조가 다름
                symbol = chunk[0]
                if not data.empty:
                    df_ticker = data.copy()
                    df_ticker['Company'] = symbol
                    df_ticker = df_ticker.reset_index()
                    all_data.append(df_ticker)
                else:
                    failed_tickers.append(symbol)
            else:
                # 멀티 티커
                for symbol in chunk:
                    try:
                        if symbol in data.columns.get_level_values(0):
                            df_ticker = data[symbol].copy()
                            df_ticker['Company'] = symbol
                            df_ticker = df_ticker.reset_index()
                            
                            # NaN 행 제거
                            df_ticker = df_ticker.dropna(subset=['Close'])
                            
                            if not df_ticker.empty:
                                all_data.append(df_ticker)
                            else:
                                failed_tickers.append(symbol)
                        else:
                            failed_tickers.append(symbol)
                    except Exception:
                        failed_tickers.append(symbol)
            
            time.sleep(0.1)  # Rate limit 방지
            
        except Exception as e:
            print(f"Batch Error: {e}")
            failed_tickers.extend(chunk)
    
    # 데이터 병합
    if all_data:
        df_combined = pd.concat(all_data, ignore_index=True)
        
        # 컬럼명 정리
        df_combined.columns = [col.replace(' ', '_') if isinstance(col, str) else col for col in df_combined.columns]
        
        # Date 컬럼 처리
        if 'Date' in df_combined.columns:
            df_combined['Date'] = pd.to_datetime(df_combined['Date']).dt.tz_localize(None)
        
        return df_combined, failed_tickers
    else:
        return pd.DataFrame(), failed_tickers

가격 데이터 수집 함수 정의 완료


In [26]:
# 수집 기간 설정
existing_max_date = df_existing['Date'].max()
start_date = (existing_max_date + timedelta(days=1)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

print(f"기존 데이터 마지막 날짜: {existing_max_date.strftime('%Y-%m-%d')}")
print(f"신규 데이터 수집 기간: {start_date} ~ {end_date}")

기존 데이터 마지막 날짜: 2023-11-29
신규 데이터 수집 기간: 2023-11-30 ~ 2026-01-12


In [ ]:
# 가격 데이터 수집 실행
print(f"대상 티커 수: {len(tickers)}개")

df_new, price_failed_tickers = fetch_price_data(tickers, start_date, end_date)

print(f"\n수집 완료")
print(f"신규 데이터 Shape: {df_new.shape}")
print(f"수집 실패 티커: {len(price_failed_tickers)}개")

if price_failed_tickers:
    print(f"\n실패 티커: {price_failed_tickers[:20]}{'...' if len(price_failed_tickers) > 20 else ''}")

대상 티커 수: 491개
예상 소요 시간: ~1.2분




Fetching Price Data:   0%|          | 0/10 [00:00<?, ?it/s]

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ANSS"}}}


$ANSS: possibly delisted; no timezone found



2 Failed downloads:


...

['SPLK', 'SQ']: possibly delisted; no timezone found



Fetching Price Data:  90%|█████████ | 9/10 [00:34<00:03,  3.77s/it]


Fetching Price Data: 100%|██████████| 10/10 [00:37<00:00,  3.54s/it]


Fetching Price Data: 100%|██████████| 10/10 [00:37<00:00,  3.71s/it]


수집 완료!
신규 데이터 Shape: (254439, 7)
수집 실패 티커: 10개

실패 티커: ['AFL', 'ANSS', 'BGNE', 'DFS', 'HES', 'ORAN', 'PXD', 'SGEN', 'SPLK', 'SQ']


In [28]:
# 신규 데이터 확인
if not df_new.empty:
    print(f"\n신규 데이터 기간: {df_new['Date'].min()} ~ {df_new['Date'].max()}")
    print(f"신규 데이터 기업 수: {df_new['Company'].nunique()}")
    print(f"\n컬럼 목록: {df_new.columns.tolist()}")
    df_new.head()


신규 데이터 기간: 2023-11-30 00:00:00 ~ 2026-01-09 00:00:00
신규 데이터 기업 수: 481

컬럼 목록: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company']


---

## Section 5. 신규 데이터 품질 검증

In [29]:
# 신규 데이터 OHLC 검증
if not df_new.empty:
    # 컬럼명 매핑 (yfinance 출력 형식에 맞춤)
    col_mapping = {
        'Adj_Close': 'Close',  # auto_adjust=True면 이미 조정됨
    }
    df_new_check = df_new.rename(columns=col_mapping)
    
    new_anomalies = validate_ohlc(df_new_check)
    
    print("=" * 60)
    print("신규 데이터 OHLC 검증")
    print("=" * 60)
    if len(new_anomalies) == 0:
        print("이상치 없음")
    else:
        print(f"총 이상치: {len(new_anomalies)}건")
        print(new_anomalies['Type'].value_counts())

신규 데이터 OHLC 검증
총 이상치: 4건
Type
Open/Close out of H/L range    4
Name: count, dtype: int64


---

## Section 6. 데이터 정합성 검증 & 병합

In [30]:
def validate_price_continuity(df_old, df_new, threshold=0.05):
    """
    기존 데이터와 신규 데이터의 연결 시점 가격 점프 검증
    
    Args:
        df_old: 기존 데이터
        df_new: 신규 데이터
        threshold: 허용 변동률 (기본 5%)
    
    Returns:
        dict: 검증 결과
    """
    warnings_list = []
    
    # 기존 데이터의 마지막 종가
    last_prices = df_old.groupby('Company').last()[['Date', 'Close']].reset_index()
    last_prices.columns = ['Company', 'Old_Date', 'Old_Close']
    
    # 신규 데이터의 첫 종가
    first_prices = df_new.groupby('Company').first()[['Date', 'Close']].reset_index()
    first_prices.columns = ['Company', 'New_Date', 'New_Close']
    
    # 병합
    merged = pd.merge(last_prices, first_prices, on='Company', how='inner')
    
    # 변동률 계산
    merged['Price_Change'] = (merged['New_Close'] - merged['Old_Close']) / merged['Old_Close']
    merged['Price_Change_Abs'] = merged['Price_Change'].abs()
    
    # 임계값 초과 검출
    suspicious = merged[merged['Price_Change_Abs'] > threshold]
    
    for _, row in suspicious.iterrows():
        warnings_list.append({
            'Company': row['Company'],
            'Old_Date': row['Old_Date'],
            'Old_Close': row['Old_Close'],
            'New_Date': row['New_Date'],
            'New_Close': row['New_Close'],
            'Change_Pct': row['Price_Change'] * 100
        })
    
    return {
        'total_companies': len(merged),
        'warnings_count': len(warnings_list),
        'warnings': pd.DataFrame(warnings_list) if warnings_list else pd.DataFrame()
    }

print("정합성 검증 함수 정의 완료")

정합성 검증 함수 정의 완료


In [31]:
# 정합성 검증 실행
if not df_new.empty:
    continuity_result = validate_price_continuity(df_existing, df_new, threshold=0.05)
    
    print("=" * 60)
    print("가격 연속성 검증 결과")
    print("=" * 60)
    print(f"검증 대상 기업: {continuity_result['total_companies']}개")
    print(f"5% 이상 가격 점프: {continuity_result['warnings_count']}개")
    
    if continuity_result['warnings_count'] > 0:
        print("\n가격 점프 상세 (상위 10건):")
        print(continuity_result['warnings'].sort_values('Change_Pct', key=abs, ascending=False).head(10).to_string(index=False))
    else:
        print("\n모든 기업 정상")

가격 연속성 검증 결과
검증 대상 기업: 481개
5% 이상 가격 점프: 198개

가격 점프 상세 (상위 10건):
Company   Old_Date   Old_Close   New_Date  New_Close  Change_Pct
    CMG 2023-11-29 2194.449951 2023-11-30  44.044998  -97.992891
   AVGO 2023-11-29  940.530029 2023-11-30  90.110336  -90.419196
   LRCX 2023-11-29  720.604980 2023-11-30  70.036964  -90.280810
   NVDA 2023-11-29  480.734985 2023-11-30  46.739845  -90.277420
   NFLX 2023-11-29  476.940002 2023-11-30  47.396999  -90.062272
     IX 2023-11-29   91.070000 2023-11-30  17.343882  -80.955439
   TSCO 2023-11-29  203.500000 2023-11-30  39.275684  -80.699909
   SONY 2023-11-29   86.305000 2023-11-30  17.026321  -80.271918
    NOW 2023-11-29  681.080017 2023-11-30 137.147995  -79.863160
   CTAS 2023-11-29  549.890015 2023-11-30 136.092163  -75.251021


In [32]:
# 데이터 병합 준비
if not df_new.empty:
    # 신규 데이터 컬럼 정리
    df_new_clean = df_new.copy()
    
    # 필요한 컬럼만 선택 (기존 데이터 구조와 맞춤)
    base_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company']
    
    # 컬럼 존재 확인 및 선택
    available_cols = [col for col in base_columns if col in df_new_clean.columns]
    df_new_clean = df_new_clean[available_cols].copy()
    
    # Dividends, Stock Splits 추가 (없으면 0)
    if 'Dividends' not in df_new_clean.columns:
        df_new_clean['Dividends'] = 0.0
    if 'Stock Splits' not in df_new_clean.columns:
        df_new_clean['Stock Splits'] = 0.0
    
    print(f"정리된 신규 데이터 Shape: {df_new_clean.shape}")
    print(f"컬럼: {df_new_clean.columns.tolist()}")

정리된 신규 데이터 Shape: (254439, 9)
컬럼: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company', 'Dividends', 'Stock Splits']


In [33]:
# 메타데이터 매핑
if not df_new.empty:
    # 신규 데이터에 섹터/산업 매핑
    df_new_clean = df_new_clean.merge(metadata_df, on='Company', how='left')
    df_new_clean['Sector'] = df_new_clean['Sector_New'].fillna('Unknown')
    df_new_clean['Industry'] = df_new_clean['Industry_New'].fillna('Unknown')
    df_new_clean = df_new_clean.drop(columns=['Sector_New', 'Industry_New'], errors='ignore')
    
    print(f"메타데이터 매핑 완료")
    print(df_new_clean.head())

메타데이터 매핑 완료
        Date        Open        High         Low       Close     Volume  \
0 2023-11-30  125.889922  126.332501  124.876903  125.693222  2442900.0   
1 2023-12-01  125.240799  126.883268  124.424487  126.666893  1729600.0   
2 2023-12-04  126.293171  127.089816  125.762072  126.755424  1543200.0   
3 2023-12-05  125.703043  126.509532  123.696672  125.771889  1933600.0   
4 2023-12-06  126.381690  127.620914  125.811250  126.784935  1816800.0   

  Company  Dividends  Stock Splits      Sector                Industry  
0       A        0.0           0.0  Healthcare  Diagnostics & Research  
1       A        0.0           0.0  Healthcare  Diagnostics & Research  
2       A        0.0           0.0  Healthcare  Diagnostics & Research  
3       A        0.0           0.0  Healthcare  Diagnostics & Research  
4       A        0.0           0.0  Healthcare  Diagnostics & Research  


In [34]:
# 기존 데이터 + 신규 데이터 병합
if not df_new.empty:
    # 기존 데이터에서 파생변수 컬럼 제거 후 재계산
    base_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Company', 'Sector']
    df_existing_base = df_existing[[col for col in base_cols if col in df_existing.columns]].copy()
    
    # 기존 데이터 섹터 업데이트
    df_existing_base = df_existing_base.drop(columns=['Sector'], errors='ignore')
    df_existing_base = df_existing_base.merge(metadata_df[['Company', 'Sector_New']], on='Company', how='left')
    df_existing_base['Sector'] = df_existing_base['Sector_New'].fillna('Unknown')
    df_existing_base = df_existing_base.drop(columns=['Sector_New'], errors='ignore')
    
    # Industry 추가
    if 'Industry_New' in metadata_df.columns:
        df_existing_base = df_existing_base.merge(metadata_df[['Company', 'Industry_New']], on='Company', how='left')
        df_existing_base['Industry'] = df_existing_base['Industry_New'].fillna('Unknown')
        df_existing_base = df_existing_base.drop(columns=['Industry_New'], errors='ignore')
    
    # 병합
    df_combined = pd.concat([df_existing_base, df_new_clean], ignore_index=True)
    
    # 중복 제거 (Company, Date 기준)
    df_combined = df_combined.drop_duplicates(subset=['Company', 'Date'], keep='last')
    
    # 정렬
    df_combined = df_combined.sort_values(['Company', 'Date']).reset_index(drop=True)
    
    print(f"\n병합 완료!")
    print(f"병합 데이터 Shape: {df_combined.shape}")
    print(f"기간: {df_combined['Date'].min()} ~ {df_combined['Date'].max()}")
    print(f"기업 수: {df_combined['Company'].nunique()}")
else:
    # 신규 데이터가 없으면 기존 데이터만 사용
    df_combined = df_existing.copy()
    print("신규 데이터 없음. 기존 데이터만 사용.")


병합 완료!
병합 데이터 Shape: (857401, 11)
기간: 2018-11-29 00:00:00 ~ 2026-01-09 00:00:00
기업 수: 491


---

## Section 7. 파생변수 계산

### 계산 대상
- **기존**: Daily_Return, Cum_Return, MA_5/20/60, Volatility_20d, MDD, Vol_Ratio, Vol_Z_Score, Gap_Pct
- **신규**: RSI_14, BB_Upper, BB_Middle, BB_Lower, BB_Width

In [35]:
def calculate_rsi(series, period=14):
    """
    RSI (Relative Strength Index) 계산
    
    Args:
        series: 종가 시리즈
        period: 기간 (기본 14일)
    
    Returns:
        RSI 시리즈
    """
    delta = series.diff()
    
    gain = delta.where(delta > 0, 0)
    loss = (-delta).where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


def calculate_bollinger_bands(series, period=20, std_dev=2):
    """
    볼린저 밴드 계산
    
    Args:
        series: 종가 시리즈
        period: 이동평균 기간 (기본 20일)
        std_dev: 표준편차 배수 (기본 2)
    
    Returns:
        DataFrame: BB_Upper, BB_Middle, BB_Lower, BB_Width
    """
    middle = series.rolling(window=period).mean()
    std = series.rolling(window=period).std()
    
    upper = middle + (std * std_dev)
    lower = middle - (std * std_dev)
    width = (upper - lower) / middle * 100  # 백분율
    
    return pd.DataFrame({
        'BB_Upper': upper,
        'BB_Middle': middle,
        'BB_Lower': lower,
        'BB_Width': width
    })

print("RSI, Bollinger Bands 함수 정의 완료")

RSI, Bollinger Bands 함수 정의 완료


In [ ]:
def calculate_all_features(df):
    """
    모든 파생변수 일괄 계산 (데이터 누수 방지 개선 버전)
    
    Args:
        df: 기본 OHLCV 데이터
    
    Returns:
        DataFrame: 파생변수가 추가된 데이터
    """
    df = df.copy()
    df = df.sort_values(['Company', 'Date']).reset_index(drop=True)
        
    # 1. 일간 수익률 (기업별로 독립 계산, 첫날은 NaN 유지)
    df['Daily_Return'] = df.groupby('Company')['Close'].pct_change()
    
    # 2. 누적 수익률 (기업별 독립 계산)
    df['Cum_Return'] = df.groupby('Company')['Close'].transform(
        lambda x: (x / x.iloc[0]) - 1
    )
    
    # 3. 이동평균선 (기업별 독립 계산)
    for window in [5, 20, 60]:
        df[f'MA_{window}'] = df.groupby('Company')['Close'].transform(
            lambda x: x.rolling(window).mean()
        )
    
    # 4. 변동성 (20일, 기업별 독립 계산)
    df['Volatility_20d'] = df.groupby('Company')['Daily_Return'].transform(
        lambda x: x.rolling(20).std()
    )
    
    # 5. MDD (Maximum Drawdown, 기업별 독립 계산)
    df['Cum_Max'] = df.groupby('Company')['Close'].transform(
        lambda x: x.expanding().max()
    )
    df['Drawdown'] = (df['Close'] - df['Cum_Max']) / df['Cum_Max']
    df['MDD'] = df.groupby('Company')['Drawdown'].transform(
        lambda x: x.expanding().min()
    )
    
    # 6. 거래량 지표 (기업별 독립 계산)
    df['Vol_MA_20'] = df.groupby('Company')['Volume'].transform(
        lambda x: x.rolling(20).mean()
    )
    df['Vol_Ratio'] = df['Volume'] / df['Vol_MA_20']
    df['Vol_Std_20'] = df.groupby('Company')['Volume'].transform(
        lambda x: x.rolling(20).std()
    )
    df['Vol_Z_Score'] = (df['Volume'] - df['Vol_MA_20']) / df['Vol_Std_20']
    
    # 7. 갭 분석 (기업별 독립 계산)
    df['Prev_Close'] = df.groupby('Company')['Close'].shift(1)
    df['Gap'] = df['Open'] - df['Prev_Close']
    df['Gap_Pct'] = (df['Open'] / df['Prev_Close']) - 1
    
    # 8. RSI (14일, 기업별 독립 계산)
    print("  - RSI_14 계산 중...")
    df['RSI_14'] = df.groupby('Company')['Close'].transform(
        lambda x: calculate_rsi(x, period=14)
    )
    
    # 9. 볼린저 밴드 (기업별 독립 계산)
    bb_results = df.groupby('Company')['Close'].apply(
        lambda x: calculate_bollinger_bands(x, period=20, std_dev=2)
    )
    
    # 볼린저 밴드 결과 병합
    bb_df = bb_results.reset_index()
    bb_df = bb_df.drop(columns=['level_1'], errors='ignore')
    
    # 인덱스 기반 병합을 위해 원본 인덱스 복원
    df = df.reset_index(drop=True)
    for col in ['BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width']:
        if col in bb_df.columns:
            df[col] = bb_df[col].values
    
    return df

전체 파생변수 계산 함수 정의 완료 (데이터 누수 방지 개선)


In [37]:
# 파생변수 계산 실행
df_final = calculate_all_features(df_combined)

print(f"\n최종 데이터 Shape: {df_final.shape}")
print(f"\n컬럼 목록:")
print(df_final.columns.tolist())

파생변수 계산 중...
  - Daily_Return 계산 중... (groupby 적용)
  - Cum_Return 계산 중...


  - MA_5, MA_20, MA_60 계산 중...


  - Volatility_20d 계산 중...


  - MDD 계산 중...
  - 거래량 지표 계산 중...


  - 갭 분석 계산 중...
  - RSI_14 계산 중...


  - Bollinger Bands 계산 중...



✅ 파생변수 계산 완료!
   - 모든 계산에 groupby('Company') 적용
   - 각 기업의 첫 거래일 Daily_Return = NaN (정상)

최종 데이터 Shape: (857401, 32)

컬럼 목록:
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Company', 'Sector', 'Industry', 'Daily_Return', 'Cum_Return', 'MA_5', 'MA_20', 'MA_60', 'Volatility_20d', 'Cum_Max', 'Drawdown', 'MDD', 'Vol_MA_20', 'Vol_Ratio', 'Vol_Std_20', 'Vol_Z_Score', 'Prev_Close', 'Gap', 'Gap_Pct', 'RSI_14', 'BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width']


In [38]:
# 파생변수 샘플 확인
sample_ticker = 'AAPL'
sample_data = df_final[df_final['Company'] == sample_ticker].tail(10)

print(f"\n{sample_ticker} 최근 10일 데이터:")
display_cols = ['Date', 'Close', 'Daily_Return', 'RSI_14', 'BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width']
print(sample_data[display_cols].to_string(index=False))


AAPL 최근 10일 데이터:
      Date      Close  Daily_Return    RSI_14   BB_Upper  BB_Middle   BB_Lower  BB_Width
2025-12-26 273.399994     -0.001497 36.148319 285.597359 276.944499 268.291640  6.248804
2025-12-29 273.760010      0.001317 39.068321 285.406065 276.689999 267.973934  6.300239
2025-12-30 273.079987     -0.002484 39.130463 284.496009 276.188998 267.881988  6.015453
2025-12-31 271.859985     -0.004468 31.277093 282.524683 275.472498 268.420312  5.120065
2026-01-02 271.010010     -0.003127 31.108797 280.836780 274.815498 268.794217  4.382054
2026-01-05 267.260010     -0.013837 25.045369 280.395090 274.143498 267.891906  4.560817
2026-01-06 262.359985     -0.018334 24.243819 281.129574 273.322498 265.515421  5.712721
2026-01-07 260.329987     -0.007737 20.665643 281.870552 272.444496 263.018440  6.919615
2026-01-08 259.040009     -0.004955 22.003583 282.422952 271.537497 260.652042  8.017644
2026-01-09 259.369995      0.001274 21.935218 282.170982 270.566997 258.963011  8.577532


---

## Section 8. 집계 데이터 재생성

In [39]:
# Sector Summary 재생성
df_sector = df_final.groupby(['Date', 'Sector']).agg({
    'Daily_Return': 'mean',
    'Close': 'mean',
    'Vol_Ratio': 'mean',
    'Volatility_20d': 'mean',
    'MDD': 'mean',
    'RSI_14': 'mean',
    'Company': 'count'
}).reset_index()

df_sector = df_sector.rename(columns={
    'Daily_Return': 'Sector_Return',
    'Close': 'Sector_Price_Avg',
    'Vol_Ratio': 'Sector_Vol_Ratio',
    'Volatility_20d': 'Sector_Volatility',
    'MDD': 'Sector_MDD',
    'RSI_14': 'Sector_RSI',
    'Company': 'Stock_Count'
})

print(f"Sector Summary Shape: {df_sector.shape}")
print(f"기간: {df_sector['Date'].min()} ~ {df_sector['Date'].max()}")
df_sector.head()

Sector Summary Shape: (20915, 9)
기간: 2018-11-29 00:00:00 ~ 2026-01-09 00:00:00


,Date,Sector,Sector_Return,Sector_Price_Avg,Sector_Vol_Ratio,Sector_Volatility,Sector_MDD,Sector_RSI,Stock_Count
0,2018-11-29,Basic Materials,NaN,69.660312,NaN,NaN,0.0,NaN,24
1,2018-11-29,Communication Services,NaN,76.977137,NaN,NaN,0.0,NaN,25
2,2018-11-29,Consumer Cyclical,NaN,214.557717,NaN,NaN,0.0,NaN,40
3,2018-11-29,Consumer Defensive,NaN,72.924028,NaN,NaN,0.0,NaN,30
4,2018-11-29,Energy,NaN,34.958204,NaN,NaN,0.0,NaN,38


In [40]:
# Monthly Summary 재생성
df_monthly = df_final[['Company', 'Sector', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
df_monthly = df_monthly.set_index('Date').groupby('Company').resample('ME').agg({
    'Sector': 'first',
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).reset_index()

# 월간 수익률
df_monthly['Monthly_Return'] = df_monthly.groupby('Company')['Close'].pct_change() * 100

# 월간 변동폭
df_monthly['Monthly_Range'] = (df_monthly['High'] - df_monthly['Low']) / df_monthly['Open'] * 100

print(f"Monthly Summary Shape: {df_monthly.shape}")
print(f"기간: {df_monthly['Date'].min()} ~ {df_monthly['Date'].max()}")
df_monthly.head()

Monthly Summary Shape: (41737, 10)
기간: 2018-11-30 00:00:00 ~ 2026-01-31 00:00:00


,Company,Date,Sector,Open,High,Low,Close,Volume,Monthly_Return,Monthly_Range
0,A,2018-11-30,Healthcare,68.673458,70.042470,68.673458,69.753235,4905300.0,NaN,1.993510
1,A,2018-12-31,Healthcare,70.698073,72.414195,59.986839,65.199600,50474000.0,-6.528206,17.578070
2,A,2019-01-31,Healthcare,64.271775,73.753060,59.922551,73.501778,44194500.0,12.733479,21.518791
3,A,2019-02-28,Healthcare,73.608101,77.164765,72.255001,76.778175,35941700.0,4.457576,6.670140
4,A,2019-03-31,Healthcare,77.319419,79.513353,74.806536,77.686691,32806600.0,1.183300,6.087497


---

## Section 9. 최종 QA 리포트 & 저장

In [41]:
# QA 요약 리포트 생성
qa_summary = {
    'execution_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'data_period': {
        'start': df_final['Date'].min().strftime('%Y-%m-%d'),
        'end': df_final['Date'].max().strftime('%Y-%m-%d')
    },
    'company_analysis': {
        'total_companies': len(tickers),
        'full_period_companies': len(full_period_companies),
        'partial_period_companies': len(partial_period_companies)
    },
    'ohlc_anomalies': len(anomaly_df) if 'anomaly_df' in dir() else 0,
    'event_companies': {
        'dividend_companies': len(dividend_companies),
        'split_companies': len(split_companies)
    },
    'new_data': {
        'rows_added': len(df_new) if not df_new.empty else 0,
        'failed_tickers': len(price_failed_tickers) if 'price_failed_tickers' in dir() else 0
    },
    'final_data': {
        'total_rows': len(df_final),
        'total_columns': len(df_final.columns)
    }
}

print("=" * 60)
print("데이터 품질 검증 리포트")
print("=" * 60)
print(f"\n실행 시간: {qa_summary['execution_time']}")
print(f"\n1. 데이터 기간")
print(f"   {qa_summary['data_period']['start']} ~ {qa_summary['data_period']['end']}")
print(f"\n2. 기업별 관측일 분석")
print(f"   - 전체 기업 수: {qa_summary['company_analysis']['total_companies']}개")
print(f"   - 동일 기간 비교 가능: {qa_summary['company_analysis']['full_period_companies']}개")
print(f"   - 개별 분석 대상: {qa_summary['company_analysis']['partial_period_companies']}개")
print(f"\n3. OHLC 이상치")
print(f"   - 발견된 이상치: {qa_summary['ohlc_anomalies']}건")
print(f"\n4. 이벤트 기업")
print(f"   - 배당 지급 기업: {qa_summary['event_companies']['dividend_companies']}개")
print(f"   - 주식 분할 기업: {qa_summary['event_companies']['split_companies']}개")
print(f"\n5. 신규 데이터")
print(f"   - 추가된 행 수: {qa_summary['new_data']['rows_added']:,}개")
print(f"   - 수집 실패 티커: {qa_summary['new_data']['failed_tickers']}개")
print(f"\n6. 최종 데이터")
print(f"   - 총 행 수: {qa_summary['final_data']['total_rows']:,}개")
print(f"   - 총 컬럼 수: {qa_summary['final_data']['total_columns']}개")
print("\n" + "=" * 60)

데이터 품질 검증 리포트

실행 시간: 2026-01-12 13:54:10

1. 데이터 기간
   2018-11-29 ~ 2026-01-09

2. 기업별 관측일 분석
   - 전체 기업 수: 491개
   - 동일 기간 비교 가능: 471개
   - 개별 분석 대상: 20개

3. OHLC 이상치
   - 발견된 이상치: 42건

4. 이벤트 기업
   - 배당 지급 기업: 385개
   - 주식 분할 기업: 61개

5. 신규 데이터
   - 추가된 행 수: 254,439개
   - 수집 실패 티커: 10개

6. 최종 데이터
   - 총 행 수: 857,401개
   - 총 컬럼 수: 32개



In [42]:
# QA 파일 저장
# 1. 동일 기간 비교 가능 기업
full_period_df = company_stats[company_stats['Company'].isin(full_period_companies)]
full_period_df.to_csv(QA_DIR / 'full_period_companies.csv', index=False)

# 2. 개별 분석 대상 기업
partial_period_df = company_stats[company_stats['Company'].isin(partial_period_companies)]
partial_period_df.to_csv(QA_DIR / 'partial_period_companies.csv', index=False)

# 3. 배당/분할 이벤트 기업
if len(event_df) > 0:
    event_df.to_csv(QA_DIR / 'dividend_split_events.csv', index=False)

# 4. OHLC 이상치 리포트
if len(anomaly_df) > 0:
    anomaly_df.to_csv(QA_DIR / 'anomaly_report.csv', index=False)

# 5. QA 요약 JSON
with open(QA_DIR / 'qa_summary.json', 'w') as f:
    json.dump(qa_summary, f, indent=2, default=str)

print("QA 파일 저장 완료:")
print(f"  - {QA_DIR / 'full_period_companies.csv'}")
print(f"  - {QA_DIR / 'partial_period_companies.csv'}")
print(f"  - {QA_DIR / 'dividend_split_events.csv'}")
print(f"  - {QA_DIR / 'anomaly_report.csv'}")
print(f"  - {QA_DIR / 'qa_summary.json'}")

QA 파일 저장 완료:
  - /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/QA/full_period_companies.csv
  - /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/QA/partial_period_companies.csv
  - /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/QA/dividend_split_events.csv
  - /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/QA/anomaly_report.csv
  - /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set/QA/qa_summary.json


In [43]:
# 메인 데이터 저장
df_final.to_csv(DATA_DIR / 'stock_daily_master.csv', index=False)
df_sector.to_csv(DATA_DIR / 'stock_sector_summary.csv', index=False)
df_monthly.to_csv(DATA_DIR / 'stock_monthly_summary.csv', index=False)

# 메타데이터 저장
metadata_df.to_csv(DATA_DIR / 'ticker_metadata.csv', index=False)

print("\n[저장 완료]")
print(f"  1. Daily Master: {df_final.shape} -> stock_daily_master.csv")
print(f"  2. Sector Summary: {df_sector.shape} -> stock_sector_summary.csv")
print(f"  3. Monthly Summary: {df_monthly.shape} -> stock_monthly_summary.csv")
print(f"  4. Ticker Metadata: {metadata_df.shape} -> ticker_metadata.csv")


[저장 완료]
  1. Daily Master: (857401, 32) -> stock_daily_master.csv
  2. Sector Summary: (20915, 9) -> stock_sector_summary.csv
  3. Monthly Summary: (41737, 10) -> stock_monthly_summary.csv
  4. Ticker Metadata: (491, 3) -> ticker_metadata.csv
